In [1]:
import pandas as pd
from sportsreference.ncaab.boxscore2 import Boxscore
from sportsreference.ncaab.teams import Team
import requests
from bs4 import BeautifulSoup
import pickle
from time import sleep
import numpy as np
pd.options.display.max_columns = None

In [2]:
def get_boxscore_links() -> set:

    url = "https://www.sports-reference.com/cbb/schools/purdue/2023-schedule.html"
    r = requests.get(url)
    if r.status_code != 200:
        raise Exception("Website is not cooperating")
    
    soup = BeautifulSoup(r.content)
    # Finding the table we need
    games = soup.findAll('table')[1]
    # Extracting strings which contain the sub-string "boxscore"
    raw_boxscore_links = [block for block in str(games.findAll('a',href = True)).split("</a>") if "boxscores" in block]

    output_set = set()

    for boxscore_link in raw_boxscore_links:

        # Extracting the particular text I want and adding it to the set
        split_string = boxscore_link.split("boxscores/")
        output_set.add(split_string[1].split('"')[0].replace('.html',""))

    return output_set

purdue_player_ids = ['fletcher-loyer-1',
                    'braden-smith-1',
                    'zach-edey-1',
                    'ethan-morton-1',
                    'mason-gillis-1',
                    'brandon-newman-2',
                    'caleb-furst-1',
                    'trey-kaufman-renn-1',
                    'david-jenkinsjr-1',
                    'brian-waddell-1',
                    'carson-barrett-1',
                    'sam-king-2',
                    'will-berg-1',
                    'chase-martin-1',
                    'matt-frost-1',
                    'camden-heide-1']

In [3]:
boxscore_links = get_boxscore_links()
with open('pickle_files/boxscore_links.pickle','wb') as f:
    pickle.dump(boxscore_links,f)

In [61]:
master_df = pd.read_csv('csvs/all_game_boxscores.csv')
for idx,boxscore_link in enumerate(boxscore_links):

    if boxscore_link in master_df.boxscore_link.tolist():
        continue

    print(boxscore_link)
    boxscore_object = Boxscore(boxscore_link)
    temp_df = boxscore_object.dataframe.copy()
    temp_df = temp_df.rename_axis('boxscore_link').reset_index()
    master_df = pd.concat([master_df,temp_df])

    if idx < len(boxscore_links) - 1:

        random_integer = np.random.randint(120,300)
        sleep(random_integer)

    master_df.to_csv('csvs/all_game_boxscores.csv',index = False)

2022-12-10-14-nebraska
2022-12-07-19-purdue
2022-12-04-18-purdue


In [29]:
player_boxscore_dataframe = pd.read_csv('csvs/player_boxscore_dataframe.csv')
for idx,boxscore_link in enumerate(boxscore_links):

    if boxscore_link in player_boxscore_dataframe.game_id.unique().tolist():
        continue

    print(boxscore_link)
    boxscore_object = Boxscore(boxscore_link)

    #Get date
    game_date = "-".join(boxscore_link.split('-')[0:3])

    for player in boxscore_object.home_players:

        temp_df = player.dataframe.rename_axis('player_id').reset_index().copy()
        temp_df['purdue_player'] = 1 if player.player_id in purdue_player_ids else 0
        temp_df['game_id'] = boxscore_link
        temp_df['game_date'] = game_date
        player_boxscore_dataframe = pd.concat([player_boxscore_dataframe,temp_df])
    
    for player in boxscore_object.away_players:

        temp_df = player.dataframe.rename_axis('player_id').reset_index().copy()
        temp_df['game_id'] = boxscore_link
        temp_df['purdue_player'] = 1 if player.player_id in purdue_player_ids else 0
        temp_df['game_date'] = game_date
        player_boxscore_dataframe = pd.concat([player_boxscore_dataframe,temp_df])
    
    if 'ethan-morton-1' not in player_boxscore_dataframe[player_boxscore_dataframe.game_id == boxscore_link].player_id.tolist():
        print(f"Morton missing from {boxscore_link}")
    
    sleep(np.random.randint(45,90))
player_boxscore_dataframe.game_date = pd.to_datetime(player_boxscore_dataframe.game_date)
player_boxscore_dataframe.sort_values('game_date').to_csv('csvs/player_boxscore_dataframe.csv',index = False)

2022-12-10-14-nebraska
2022-11-24-purdue
2022-11-11-purdue
2022-11-25-23-gonzaga
2022-11-27-15-purdue
2022-11-08-purdue
2022-11-15-purdue
2022-12-04-18-purdue
2022-11-30-00-florida-state
2022-12-07-19-purdue
